# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [118]:
#%pip install iso3166
#%pip install seaborn

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [119]:
#%pip install --upgrade plotly
#%pip install --upgrade nbformat

### Import Statements

In [120]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [121]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [122]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [123]:
df_data.shape

(4324, 9)

In [124]:
df_data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Organisation', 'Location', 'Date',
       'Detail', 'Rocket_Status', 'Price', 'Mission_Status'],
      dtype='object')

In [125]:
df_data.isna().any()

Unnamed: 0.1      False
Unnamed: 0        False
Organisation      False
Location          False
Date              False
Detail            False
Rocket_Status     False
Price              True
Mission_Status    False
dtype: bool

In [126]:
df_data.duplicated().any()

False

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [127]:
df_data.drop(['Unnamed: 0.1','Unnamed: 0'], axis=1, inplace=True)

## Descriptive Statistics

In [128]:
df_data.describe()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
count,4324,4324,4324,4324,4324,964,4324
unique,56,137,4319,4278,2,56,4
top,RVSN USSR,"Site 31/6, Baikonur Cosmodrome, Kazakhstan","Wed Nov 05, 2008 00:15 UTC",Cosmos-3MRB (65MRB) | BOR-5 Shuttle,StatusRetired,450.0,Success
freq,1777,235,2,6,3534,136,3879


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [129]:
df_org = df_data['Organisation'].value_counts()
fig = px.bar(df_org, 
        x=df_org.index.array,
        y='Organisation',
        title='Number of launches per Company',
        labels= {
            'y': 'Launches',
            'x': 'Organisation'
        })
fig.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [130]:
df_r_status = df_data['Rocket_Status'].value_counts()
df_r_status

StatusRetired    3534
StatusActive      790
Name: Rocket_Status, dtype: int64

In [131]:
fig = px.pie(df_r_status, 
        values='Rocket_Status',
        names=df_r_status.index.array,
        title='Active versus Retired Rockets')
fig.show()

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [132]:
df_m_status = df_data['Mission_Status'].value_counts()
df_m_status

Success              3879
Failure               339
Partial Failure       102
Prelaunch Failure       4
Name: Mission_Status, dtype: int64

In [133]:
fig = px.pie(df_m_status, 
        values='Mission_Status',
        names=df_m_status.index.array,
        title='Mission Status')
fig.show()

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [134]:
df_p = df_data.dropna()
df_p_n = pd.to_numeric(df_p['Price'].str.replace(',',''))
df_p.insert(7, 'Price_N', df_p_n)


In [135]:
fig = px.histogram(df_p['Price_N'], nbins=50,
        labels= {
            'value': 'millions USD'
        })
fig.show()

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA

Location: Pacific Ocean => Kiritimati


You can use the iso3166 package to convert the country names to Alpha3 format.

# My Notes:
* Location: Pacific Ocean => Kiritimati
* [Countriy names mapping in iso3166](https://github.com/deactivated/python-iso3166/blob/master/src/iso3166/__init__.py)

In [136]:
# Split location to extra country information
df_loc = df_data['Location'].str.split(', ').str[-1]

# Map country names to ISO code
arr_loc_alpha3 = []
for item in df_loc.array:
    if item == 'Russia' or item == 'Barents Sea':
        arr_loc_alpha3.append(countries.get('Russian Federation').alpha3)
    # Map Kiritimati to USA
    elif item == 'USA' or item == 'Gran Canaria' or item == 'New Mexico' or 'Pacific' in item:
        arr_loc_alpha3.append('USA')
    elif item == 'Yellow Sea':
        arr_loc_alpha3.append(countries.get('China').alpha3)
    elif item == 'Iran' or item == 'Shahrud Missile Test Site':
        arr_loc_alpha3.append(countries.get('Iran, Islamic Republic of').alpha3)
    elif item == 'North Korea':
        arr_loc_alpha3.append(countries.get('Korea, Democratic People\'s Republic of').alpha3)
    elif item == 'South Korea':
        arr_loc_alpha3.append(countries.get('Korea, Republic of').alpha3)
    else:
        arr_loc_alpha3.append(countries.get(item).alpha3)

arr_loc_name = [countries.get(item).apolitical_name for item in arr_loc_alpha3]


In [137]:
df_data.insert(7, column='ISO', value=arr_loc_alpha3)
df_data.insert(8, column='Country', value=arr_loc_name)
df_data.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,ISO,Country
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,USA,United States of America
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,CHN,China
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,USA,United States of America
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,KAZ,Kazakhstan
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,USA,United States of America


In [138]:
df_country_info = pd.DataFrame({'ISO': df_data['ISO'],'Country': df_data['Country']})
df_country_counts = df_country_info.value_counts()
df_country_counts = df_country_counts.reset_index()
df_country_counts.columns = ['ISO','Country','Counts']
df_country_counts.head()

,ISO,Country,Counts
0,RUS,Russian Federation,1398
1,USA,United States of America,1387
2,KAZ,Kazakhstan,701
3,FRA,France,303
4,CHN,China,269


In [139]:
fig = px.choropleth(df_country_counts, locations='ISO',
                    color='Counts',
                    hover_name='Country',
                    color_continuous_scale=px.colors.sequential.matter,
                    title='Number of Launches by Country',
                    labels= {
                        'Counts': 'Number of Launches'
                        }
                    )
fig.show()


# Use a Choropleth Map to Show the Number of Failures by Country


In [140]:
df_mission_status = df_data.groupby(['Mission_Status'])['ISO'].value_counts()
df_mission_status = df_mission_status.reset_index(level=0)
df_mission_status.columns = ['Mission_Status', 'Mission_Status_Count']
df_mission_status = df_mission_status.reset_index()
df_mission_status_failure = df_mission_status[df_mission_status['Mission_Status']=='Failure']
df_mission_status_failure.head()

,ISO,Mission_Status,Mission_Status_Count
0,USA,Failure,132
1,KAZ,Failure,72
2,RUS,Failure,63
3,CHN,Failure,19
4,FRA,Failure,13


In [141]:
fig = px.choropleth(df_mission_status_failure, locations='ISO',
                    color='Mission_Status_Count',
                    hover_name='ISO',
                    color_continuous_scale=px.colors.sequential.matter,
                                        title='Number of Failures by Country',
                    labels= {
                        'Mission_Status_Count': 'Number of Failures'
                        }
                    )
fig.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [142]:
df_mission_status.head(20)

,ISO,Mission_Status,Mission_Status_Count
0,USA,Failure,132
1,KAZ,Failure,72
2,RUS,Failure,63
3,CHN,Failure,19
4,FRA,Failure,13
5,JPN,Failure,10
6,IND,Failure,8
7,IRN,Failure,8
8,AUS,Failure,3
9,PRK,Failure,3


In [143]:
fig = px.sunburst(
    df_mission_status,
    path=['ISO', 'Mission_Status'], 
    values='Mission_Status_Count'
)
fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [144]:
# Rows with NaN Price are removed, See [17]
df_p.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Price_N
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,50.00
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,29.75
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,65.00
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,145.00
5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC","Long March 4B | Ziyuan-3 03, Apocalypse-10 & N...",StatusActive,64.68,Success,64.68


In [145]:
df_p.groupby('Organisation').aggregate({'Price_N': pd.Series.sum}).sort_values(by='Price_N', ascending=False)

,Price_N
Organisation,
NASA,"76,280.00"
Arianespace,"16,345.00"
ULA,"14,798.00"
RVSN USSR,"10,000.00"
CASC,"6,340.26"
SpaceX,"5,444.00"
Northrop,"3,930.00"
MHI,"3,532.50"
ISRO,"2,177.00"


# Analyse the Amount of Money Spent by Organisation per Launch

In [146]:
# TODO: Don't understand what should be look for, the data row is per launch already? *mean?
df_p.groupby(['Organisation','Price_N']).sample()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Price_N
189,Arianespace,"ELV-1 (SLV), Guiana Space Centre, French Guian...","Wed Nov 21, 2018 01:42 UTC",Vega | Mohammed VI-B,StatusActive,37.0,Success,37.00
54,Arianespace,"Site 31/6, Baikonur Cosmodrome, Kazakhstan","Thu Feb 06, 2020 21:42 UTC",Soyuz 2.1b/Fregat | OneWeb #2,StatusActive,48.5,Success,48.50
626,Arianespace,"ELS, Guiana Space Centre, French Guiana, France","Sun Dec 02, 2012 02:02 UTC",Soyuz ST-A/Fregat | Pl??iades 1B,StatusActive,80.0,Success,80.00
1008,Arianespace,"ELA-3, Guiana Space Centre, French Guiana, France","Tue Mar 02, 2004 07:17 UTC",Ariane 5 G+ | Rosetta & Philae,StatusRetired,190.0,Success,190.00
535,Arianespace,"ELA-3, Guiana Space Centre, French Guiana, France","Thu Oct 16, 2014 21:43 UTC","Ariane 5 ECA | Intelsat 30, ARSAT-1",StatusActive,200.0,Success,200.00
...,...,...,...,...,...,...,...,...
300,VKS RF,"Site 43/4, Plesetsk Cosmodrome, Russia","Sat Dec 02, 2017 10:43 UTC",Soyuz 2.1b | Cosmos 2524,StatusActive,35.0,Success,35.00
435,VKS RF,"Site 133/3, Plesetsk Cosmodrome, Russia","Sat Jun 04, 2016 14:00 UTC",Rokot/Briz KM | Cosmos 2517,StatusRetired,41.8,Success,41.80
73,VKS RF,"Site 43/3, Plesetsk Cosmodrome, Russia","Wed Dec 11, 2019 08:54 UTC",Soyuz 2.1b/Fregat | Cosmos 2544,StatusActive,48.5,Success,48.50
177,VKS RF,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Fri Dec 21, 2018 00:20 UTC",Proton-M/Briz-M | Blagovest No.13L,StatusActive,65.0,Success,65.00


# Chart the Number of Launches per Year

In [147]:
df_data['Date'] = pd.to_datetime(df_data['Date'], utc=True)

In [148]:
df_per_year = df_data['Date'].dt.year.value_counts().sort_index()
fig = px.line(df_per_year, 
        x=df_per_year.index.array,
        y='Date',
        title='Number of Launches per Year',
        labels= {
            'Date': 'Launches',
            'x': 'Year'
        })
fig.show()


# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

In [149]:
df_per_month = df_data.groupby(df_data['Date'].dt.month)['Date'].count()
df_per_month

Date
1     268
2     336
3     353
4     383
5     326
6     402
7     351
8     373
9     365
10    381
11    336
12    450
Name: Date, dtype: int64

In [150]:
fig = px.line(df_per_month, 
        x=df_per_month.index.array,
        y='Date',
        title='Number of Launches per Month',
        labels= {
            'Date': 'Launches',
            'x': 'Year'
        })
fig.show()

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

In [151]:
df_per_month[df_per_month == df_per_month.max()]

Date
12    450
Name: Date, dtype: int64

In [152]:
df_per_month[df_per_month == df_per_month.min()]

Date
1    268
Name: Date, dtype: int64

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

In [153]:
# Using df_p which has Price_N column
df_price_over_time = df_p.groupby(df_data['Date'].dt.year).aggregate({'Price_N': pd.Series.mean})

In [154]:
fig = px.line(df_price_over_time, 
        x=df_price_over_time.index.array,
        y='Price_N',
        title='Average Price over Time',
        labels= {
            'Price_N': 'Average Price (Yearl)',
            'x': 'Year'
        })
fig.show()

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

In [155]:
# Filter top 10 Organizations
top_10 = df_data.groupby('Organisation').count()['ISO'].sort_values(ascending=False).head(10).reset_index()['Organisation']
df_top_10 = df_data[df_data['Organisation'].isin(top_10)]

In [156]:
# Group by date data
df_top_10_dt = df_top_10.groupby(['Date','Organisation']).count().reset_index()
df_top_10_dt_year = df_top_10_dt.groupby([df_top_10_dt['Date'].dt.year, 'Organisation']).count()
df_top_10_dt_year.reset_index(level=1, inplace=True)

In [157]:
fig = px.line(df_top_10_dt_year,
    x=df_top_10_dt_year.index.array, 
    y='Location', # Any column with count data
    color='Organisation',
    markers=True,
    title='Number of Launches over Time by the Top 10 Organisations',
    labels= {
        'Location': 'Number of Launches',
        'x': 'Year'
        })
fig.update_layout(xaxis=dict(range=[1957,2020]))
fig.show()

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

In [158]:
df_usa_ussr = df_data[df_data['ISO'].isin(['USA', 'RUS', 'KAZ'])]
# Don't use df_usa_ussr['ISO'] = df_usa_ussr['ISO'].str.replace('KAZ', 'ISO')
df_usa_ussr.loc[df_usa_ussr['ISO'] == 'KAZ', 'ISO'] = 'RUS'
df_usa_ussr_dt = df_usa_ussr.groupby(['Date','ISO']).count().reset_index()
df_usa_ussr_dt_year = df_usa_ussr_dt.groupby([df_usa_ussr_dt['Date'].dt.year, 'ISO']).count()
df_usa_ussr_dt_year.reset_index(level=1, inplace=True)

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

In [159]:
fig = px.pie(df_usa_ussr_dt_year, 
        values='Location',
        names='ISO',
        title='Total launches of USSR and USA')
fig.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [160]:
fig = px.line(df_usa_ussr_dt_year,
    x=df_usa_ussr_dt_year.index.array, 
    y='Location', # Any column with count data
    color='ISO',
    markers=True,
    title='Cold War Space Race',
    labels= {
        'Location': 'Number of Launches',
        'x': 'Year',
        'ISO': 'Country'
        })
fig.update_layout(xaxis=dict(range=[1957,1991]))
fig.show()

## Chart the Total Number of Mission Failures Year on Year.

In [161]:
df_status_dt = df_data.groupby(['Date','Mission_Status'], as_index=False).count()
df_status_dt_year = df_status_dt.groupby([df_status_dt['Date'].dt.year, 'Mission_Status']).count()
df_status_dt_year.reset_index(level=1, inplace=True)
df_status_dt_year.drop(['Date'], axis=1, inplace=True)
df_status_dt_year.reset_index(inplace=True)

In [162]:
# Ignore partial failures
df_status_fail_yr_count = df_status_dt_year[df_status_dt_year['Mission_Status']=='Failure'].drop(['Mission_Status','Organisation','Location','Detail','Rocket_Status','Price','Country'], axis=1)
df_status_fail_yr_count.rename(columns={'ISO': 'Failure'}, inplace=True)
fig = px.line(df_status_fail_yr_count,
    x='Date',
    y='Failure',
    markers=True,
    title='Mission Failures',
    labels= {
        'Failure': 'Number of Failure Launches',
        'Date': 'Year'
        })
fig.update_layout(xaxis=dict(range=[1957,2020]))
fig.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

In [163]:
df_status_success_yr_count = df_status_dt_year[df_status_dt_year['Mission_Status']=='Success'].drop(['Mission_Status','Organisation','Location','Detail','Rocket_Status','Price','Country'], axis=1)
df_status_success_yr_count.rename(columns={'ISO': 'Success'}, inplace=True)
df_merged_status = pd.merge(df_status_success_yr_count, df_status_fail_yr_count, on='Date')
df_merged_status['Percentage'] = df_merged_status['Failure'] / (df_merged_status['Failure'] + df_merged_status['Success']) * 100

fig = px.line(df_merged_status,
    x='Date', 
    y='Percentage',
    markers=True,
    title='Percentage of Failures over Time',
    labels= {
        'Date': 'Year'
        })
fig.update_layout(xaxis=dict(range=[1957,2020]))
fig.show()

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

In [164]:
df_all_stat = df_data.groupby(['Date','ISO','Mission_Status'], as_index=False).count().drop(['Organisation','Location','Rocket_Status','Price','Country'], axis=1)
# Get total launches
df_all_stat_yr_sum = df_all_stat.groupby([df_all_stat.Date.dt.year, 'ISO']).count().drop(['Date','Detail'], axis=1).reset_index()
df_all_stat_yr_sum.rename(columns={'Mission_Status':'Total Launches'}, inplace=True)
df_all_stat_yr_sum.head()

,Date,ISO,Total Launches
0,1957,KAZ,2
1,1957,USA,1
2,1958,KAZ,5
3,1958,USA,23
4,1959,KAZ,4


In [165]:
fig = px.line(df_all_stat_yr_sum,
    x='Date', 
    y='Total Launches',
    color ='ISO',
    markers=True,
    title='Total launches by Country over Time',
    labels= {
        'Date': 'Year'
        })
fig.update_layout(xaxis=dict(range=[1957,2020]))
fig.show()

In [166]:
# Get success launches
df_all_stat_yr = df_all_stat.groupby([df_all_stat.Date.dt.year, 'ISO', 'Mission_Status']).count().reset_index(level=[1,2])
df_stat_success_yr = df_all_stat_yr[df_all_stat_yr['Mission_Status']=='Success'].drop(['Detail'], axis=1)
df_stat_success_yr.rename(columns={'Date':'Success Launches'}, inplace=True)
df_stat_success_yr.reset_index(inplace=True)
df_stat_success_yr.head()

,Date,ISO,Mission_Status,Success Launches
0,1957,KAZ,Success,2
1,1958,KAZ,Success,1
2,1958,USA,Success,5
3,1959,KAZ,Success,2
4,1959,USA,Success,6


In [167]:
fig = px.line(df_stat_success_yr,
    x='Date', 
    y='Success Launches',
    color ='ISO',
    markers=True,
    title='Success launches by Country over Time',
    labels= {
        'Date': 'Year'
        })
fig.update_layout(xaxis=dict(range=[1957,2020]))
fig.show()

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 

In [168]:
df_org_stat = df_data.groupby(['Date','Organisation'], as_index=False).count()
df_org_stat_yr = df_org_stat.groupby([df_org_stat.Date.dt.year, 'Organisation']).count()
df_org_stat_yr.drop(['Date','Location','Detail','Rocket_Status','Price','Mission_Status','Country'], axis=1, inplace=True)
df_org_stat_yr.reset_index(level=[0,1], inplace=True)
df_org_stat_yr.rename(columns={'Date':'Year','ISO':'Launches'}, inplace=True)
df_org_stat_yr.head()

,Year,Organisation,Launches
0,1957,RVSN USSR,2
1,1957,US Navy,1
2,1958,AMBA,7
3,1958,NASA,2
4,1958,RVSN USSR,5


In [169]:
fig = px.line(df_org_stat_yr,
    x='Year', 
    y='Launches',
    color ='Organisation',
    markers=True,
    title='Launches by Organisation over Time',
)
fig.update_layout(xaxis=dict(range=[1957,2020]))
fig.show()

In [170]:
fig = px.line(df_org_stat_yr,
    x='Year', 
    y='Launches',
    color ='Organisation',
    markers=True,
    title='Launches by Organisation Recently',
)
fig.update_layout(xaxis=dict(range=[2018,2020]),yaxis=dict(range=[0,40]))
fig.show()